In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)

In [2]:
## get the variables

PATH_1=os.path.join('..','..','..','Weather_Data','ERA5','2012','era_2012.nc')
humidity_data = xr.open_dataset(PATH_1) ## variables
humidity_data

<xarray.Dataset> Size: 544MB
Dimensions:         (valid_time: 2928, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 23kB 2012-01-01 ... 2012-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
    expver          (valid_time) <U4 47kB ...
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 272MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 272MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:49 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
humidity_data= humidity_data.drop_vars('number') ## not sure what these two are
humidity_data= humidity_data.drop_vars('expver')
humidity_data

<xarray.Dataset> Size: 543MB
Dimensions:         (valid_time: 2928, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * valid_time      (valid_time) datetime64[ns] 23kB 2012-01-01 ... 2012-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 272MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 272MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:49 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
humidity_data.to_dataframe()

r         q
valid_time          pressure_level latitude longitude                     
2012-01-01 00:00:00 1000.0         49.25    -124.85    84.012985  0.003657
                                            -124.60    79.378220  0.003602
                                            -124.35    78.753220  0.003559
                                            -124.10    79.983688  0.003523
                                            -123.85    78.544235  0.003539
...                                                          ...       ...
2012-12-31 21:00:00 1000.0         24.50    -68.10     63.521248  0.008528
                                            -67.85     63.376717  0.008522
                                            -67.60     63.101326  0.008516
                                            -67.35     62.716560  0.008500
                                            -67.10     62.302498  0.008477

[67929600 rows x 2 columns]

In [5]:
## resample the data to 1 year average

year_avg = humidity_data.resample(valid_time='1YE').mean(dim='valid_time')
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2012-12-31
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:49 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
year_avg.to_dataframe()

r         q
valid_time pressure_level latitude longitude                     
2012-12-31 1000.0         49.25    -124.85    82.836456  0.005956
                                   -124.60    80.069099  0.005967
                                   -124.35    77.650093  0.005871
                                   -124.10    75.539124  0.005789
                                   -123.85    72.923378  0.005816
...                                                 ...       ...
                          24.50    -68.10     76.898781  0.014132
                                   -67.85     76.886162  0.014121
                                   -67.60     76.900238  0.014113
                                   -67.35     76.908188  0.014103
                                   -67.10     76.928627  0.014094

[23200 rows x 2 columns]

In [7]:
year_avg = year_avg.squeeze(dim="pressure_level") ## drop the pressure level dimension
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, latitude: 100, longitude: 232)
Coordinates:
    pressure_level  float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2012-12-31
Data variables:
    r               (valid_time, latitude, longitude) float32 93kB 82.84 ... ...
    q               (valid_time, latitude, longitude) float32 93kB 0.005956 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:49 GRIB to CDM+CF via cfgrib-0.9.1...

In [9]:
SHAPE_PATH=os.path.join('..','..','..','Shapefiles','county_shapefiles','2012_county_shapefile','tl_2012_us_county.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,15,001,00365280,15001,Hawaii,Hawaii County,06,H1,G4020,None,25900,None,A,10433603617,2739477738,+19.5977643,-155.5024434,"POLYGON ((-155.96346 19.08207, -155.96649 19.0..."
1,05,027,00063761,05027,Columbia,Columbia County,06,H1,G4020,None,31620,None,A,1984070931,1809509,+33.2230377,-093.2328433,"POLYGON ((-93.49052 33.01846, -93.49052 33.018..."
2,38,011,01035619,38011,Bowman,Bowman County,06,H1,G4020,None,None,None,A,3009063806,13587978,+46.1101438,-103.5059433,"POLYGON ((-104.04579 46.11943, -104.04579 46.1..."
3,36,031,00974114,36031,Essex,Essex County,06,H1,G4020,None,None,None,A,4647119053,315962259,+44.1089711,-073.7775726,"POLYGON ((-73.90969 44.42970, -73.89728 44.430..."
4,35,007,00929117,35007,Colfax,Colfax County,06,H1,G4020,None,None,None,A,9733550626,26310378,+36.6129625,-104.6401256,"POLYGON ((-104.88900 36.26114, -104.90147 36.2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3229,32,003,00863600,32003,Clark,Clark County,06,H1,G4020,332,29820,None,A,20438872004,438784795,+36.2142360,-115.0138188,"POLYGON ((-115.84599 35.96371, -115.84554 35.9..."
3230,04,017,00042808,04017,Navajo,Navajo County,06,H1,G4020,None,43320,None,A,25771507428,24078082,+35.3909336,-110.3209076,"POLYGON ((-110.75068 34.27560, -110.75068 34.2..."
3231,17,183,01785114,17183,Vermilion,Vermilion County,06,H1,G4020,None,19180,None,A,2326763515,7534496,+40.1867398,-087.7267715,"POLYGON ((-87.52801 40.38851, -87.52804 40.386..."
3232,17,181,01785113,17181,Union,Union County,06,H1,G4020,None,None,None,A,1070855764,22502294,+37.4752071,-089.2528079,"POLYGON ((-89.15367 37.60047, -89.14902 37.600..."


In [10]:
geometry_column=county_gdf['geometry']
geometry_column

0       POLYGON ((-155.96346 19.08207, -155.96649 19.0...
1       POLYGON ((-93.49052 33.01846, -93.49052 33.018...
2       POLYGON ((-104.04579 46.11943, -104.04579 46.1...
3       POLYGON ((-73.90969 44.42970, -73.89728 44.430...
4       POLYGON ((-104.88900 36.26114, -104.90147 36.2...
                              ...                        
3229    POLYGON ((-115.84599 35.96371, -115.84554 35.9...
3230    POLYGON ((-110.75068 34.27560, -110.75068 34.2...
3231    POLYGON ((-87.52801 40.38851, -87.52804 40.386...
3232    POLYGON ((-89.15367 37.60047, -89.14902 37.600...
3233    POLYGON ((-117.63097 35.24736, -117.63117 35.2...
Name: geometry, Length: 3234, dtype: geometry

In [11]:
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x          y
0    0    -155.963465  19.082072
     1    -155.966489  19.088480
     2    -155.968892  19.093265
     3    -155.972538  19.101382
     4    -155.974041  19.107789
...               ...        ...
3233 5787 -117.631018  35.214576
     5788 -117.630863  35.222860
     5789 -117.630941  35.243498
     5790 -117.630971  35.247338
     5791 -117.630973  35.247365

[7373702 rows x 2 columns]

In [12]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.  
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            county_df: Dataframe consisting the values of the variables interpolated in the county.
    ''' 
    longitude= lat_lon.loc[(var), 'x']  # get the longitude 
    extract_val=len(longitude)        ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    
    latitude= lat_lon.loc[(var), 'y']   # get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= year_avg.interp(longitude=lon_list, latitude=lat_list)
    year_avg_finer
    
    df = year_avg_finer.squeeze(dim="valid_time") ## remove the valid_time dimension, note that this will appear as a column
                                                  ## after removed

    ## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=df.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)

    initial_df=pd.concat(get_vals)
    initial_df=initial_df.drop('valid_time', axis=1) ## remove the valid_time column 
    
    mean=initial_df.mean() ## find the average of the 100 (or less) values
    average_df = pd.DataFrame(mean).T 

    county_df= average_df.convert_dtypes() ## the dtype of the mean values will be object, so convert that to float
    county_df
    
    return county_df

In [13]:
%%time

## include all counties

df_list=[]

for i in range(0,3234): ## A total of 3234 counties. index in shape file starts from 0 and ends in 3232
    try:
        df_list.append(single_county(i))
    except:
        pass

CPU times: total: 6min 17s
Wall time: 6min 22s


In [14]:
concatenate_df=pd.concat(df_list)
final_df=concatenate_df.reset_index(drop=True)
final_df = final_df.drop('pressure_level', axis=1)
final_df

,r,q
0,<NA>,<NA>
1,63.458392,0.009817
2,54.543481,0.004738
3,73.182915,0.005858
4,46.598955,0.004082
...,...,...
3229,29.287446,0.003878
3230,37.856534,0.004221
3231,64.067029,0.006957
3232,60.838061,0.007803


In [15]:
## lets first create a fips column in the original shape file

## lets create the fips column in the dataframe which is the sum of state and county both as dtype string.
## Note: not a numerical sum
## This step is necessary because there can be several counties with idential names. 

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

STATEFP       object
COUNTYFP      object
COUNTYNS      object
GEOID         object
NAME          object
NAMELSAD      object
LSAD          object
CLASSFP       object
MTFCC         object
CSAFP         object
CBSAFP        object
METDIVFP      object
FUNCSTAT      object
ALAND          int64
AWATER         int64
INTPTLAT      object
INTPTLON      object
geometry    geometry
fips           int32
dtype: object

In [17]:
county_gdf = county_gdf.drop(['STATEFP','COUNTYFP','COUNTYNS','GEOID','NAMELSAD','LSAD','CLASSFP','MTFCC'
                             ,'CSAFP','CBSAFP','METDIVFP','FUNCSTAT','ALAND','AWATER','INTPTLAT',
                             'INTPTLON'], axis=1)
county_gdf

,NAME,geometry,fips
0,Hawaii,"POLYGON ((-155.96346 19.08207, -155.96649 19.0...",15001
1,Columbia,"POLYGON ((-93.49052 33.01846, -93.49052 33.018...",5027
2,Bowman,"POLYGON ((-104.04579 46.11943, -104.04579 46.1...",38011
3,Essex,"POLYGON ((-73.90969 44.42970, -73.89728 44.430...",36031
4,Colfax,"POLYGON ((-104.88900 36.26114, -104.90147 36.2...",35007
...,...,...,...
3229,Clark,"POLYGON ((-115.84599 35.96371, -115.84554 35.9...",32003
3230,Navajo,"POLYGON ((-110.75068 34.27560, -110.75068 34.2...",4017
3231,Vermilion,"POLYGON ((-87.52801 40.38851, -87.52804 40.386...",17183
3232,Union,"POLYGON ((-89.15367 37.60047, -89.14902 37.600...",17181


In [18]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,r,q
0,Hawaii,"POLYGON ((-155.96346 19.08207, -155.96649 19.0...",15001,<NA>,<NA>
1,Columbia,"POLYGON ((-93.49052 33.01846, -93.49052 33.018...",5027,63.458392,0.009817
2,Bowman,"POLYGON ((-104.04579 46.11943, -104.04579 46.1...",38011,54.543481,0.004738
3,Essex,"POLYGON ((-73.90969 44.42970, -73.89728 44.430...",36031,73.182915,0.005858
4,Colfax,"POLYGON ((-104.88900 36.26114, -104.90147 36.2...",35007,46.598955,0.004082
...,...,...,...,...,...
3229,Clark,"POLYGON ((-115.84599 35.96371, -115.84554 35.9...",32003,29.287446,0.003878
3230,Navajo,"POLYGON ((-110.75068 34.27560, -110.75068 34.2...",4017,37.856534,0.004221
3231,Vermilion,"POLYGON ((-87.52801 40.38851, -87.52804 40.386...",17183,64.067029,0.006957
3232,Union,"POLYGON ((-89.15367 37.60047, -89.14902 37.600...",17181,60.838061,0.007803


In [19]:
county_df=merged_df.dropna()
county_df

,NAME,geometry,fips,r,q
1,Columbia,"POLYGON ((-93.49052 33.01846, -93.49052 33.018...",5027,63.458392,0.009817
2,Bowman,"POLYGON ((-104.04579 46.11943, -104.04579 46.1...",38011,54.543481,0.004738
3,Essex,"POLYGON ((-73.90969 44.42970, -73.89728 44.430...",36031,73.182915,0.005858
4,Colfax,"POLYGON ((-104.88900 36.26114, -104.90147 36.2...",35007,46.598955,0.004082
5,Sanders,"POLYGON ((-115.15693 47.44804, -115.15930 47.4...",30089,65.925251,0.004406
...,...,...,...,...,...
3229,Clark,"POLYGON ((-115.84599 35.96371, -115.84554 35.9...",32003,29.287446,0.003878
3230,Navajo,"POLYGON ((-110.75068 34.27560, -110.75068 34.2...",4017,37.856534,0.004221
3231,Vermilion,"POLYGON ((-87.52801 40.38851, -87.52804 40.386...",17183,64.067029,0.006957
3232,Union,"POLYGON ((-89.15367 37.60047, -89.14902 37.600...",17181,60.838061,0.007803


In [20]:
type(county_df)

geopandas.geodataframe.GeoDataFrame

In [21]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

,NAME,fips,r,q
1,Columbia,5027,63.458392023194982,0.009817142355750
2,Bowman,38011,54.543480820647616,0.004737966878461
3,Essex,36031,73.182915261708516,0.005858388445408
4,Colfax,35007,46.598955246820168,0.004081626667071
5,Sanders,30089,65.925250826324685,0.004406305217424
...,...,...,...,...
3229,Clark,32003,29.287445873588876,0.003877834293340
3230,Navajo,4017,37.856533980516062,0.004221045863442
3231,Vermilion,17183,64.067028859206019,0.006956916629470
3232,Union,17181,60.838060581482495,0.007802537328043


In [22]:
county_var = county_var.sort_values(by='fips')
county_var

,NAME,fips,r,q
1748,Autauga,1001,67.154796810361091,0.010010712977142
2845,Baldwin,1003,71.723462955508424,0.011565307484221
1439,Barbour,1005,67.826609843494339,0.010190003610489
1429,Bibb,1007,66.654437193139088,0.009825882937522
1466,Blount,1009,68.555797732760027,0.009578361855791
...,...,...,...,...
2068,Sweetwater,56037,48.330270313526583,0.003474028366233
2159,Teton,56039,61.530817646890789,0.003764593871319
2113,Uinta,56041,54.256222189060594,0.003454303898350
2081,Washakie,56043,44.247479052281172,0.003586091695843


In [23]:
county_var.to_pickle('era_2012.pkl')